In [15]:
import os
import re
import cv2
import gc
from tqdm import tqdm
import numpy as np

In [2]:
def verify_paths(real_video_dir, fake_video_dir):
    """Verify that the paths exist and contain videos."""
    if not os.path.exists(real_video_dir):
        raise ValueError(f"Real video directory not found: {real_video_dir}")
    if not os.path.exists(fake_video_dir):
        raise ValueError(f"Fake video directory not found: {fake_video_dir}")

    real_videos = [f for f in os.listdir(real_video_dir) if f.endswith('.mp4')]
    if not real_videos:
        raise ValueError(f"No MP4 files found in real video directory: {real_video_dir}")

    print(f"Found {len(real_videos)} real videos.")
    return real_videos


In [3]:
# def create_directory_structure():
#     """Create the required directory structure for organizing the frames."""
#     base_dir = "FF++Snaps"
#     splits = ["train", "test", "validate"]
#     categories = ["real", "fake"]

#     # Create base directory
#     if not os.path.exists(base_dir):
#         os.makedirs(base_dir)

#     # Create split and category subdirectories
#     for split in splits:
#         split_path = os.path.join(base_dir, split)
#         if not os.path.exists(split_path):
#             os.makedirs(split_path)
#         for category in categories:
#             category_path = os.path.join(split_path, category)
#             if not os.path.exists(category_path):
#                 os.makedirs(category_path)

#     return base_dir
def create_directory_structure(base_dir="FF++Snaps"):
    """Create the required directory structure for organizing the frames."""
    splits = ["train", "test", "validate"]
    categories = ["real", "fake"]
    
    # Create base directory
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
    
    # Create split and category subdirectories
    for split in splits:
        split_path = os.path.join(base_dir, split)
        if not os.path.exists(split_path):
            os.makedirs(split_path)
        for category in categories:
            category_path = os.path.join(split_path, category)
            if not os.path.exists(category_path):
                os.makedirs(category_path)
    
    return base_dir



In [4]:
# def extract_core_name(video_name):
#     """Extract the core part of the video name (ID and descriptor)."""
#     core_name = re.sub(r'__[^_]+\\.mp4$', '.mp4', video_name)  # Remove suffix after the second double underscore
#     print(f"Original: {video_name}, Core: {core_name}")  # Debugging line
#     return core_name


# def find_matching_fake_video(real_video_name, fake_video_dir):
#     """Find the matching fake video for a given real video."""
#     real_core_name = extract_core_name(real_video_name)
#     fake_videos = [f for f in os.listdir(fake_video_dir) if f.endswith('.mp4')]
    
#     print(f"Real video core: {real_core_name}")  # Debugging line
#     print(f"Fake videos: {fake_videos}")  # Debugging line

#     for fake_video_name in fake_videos:
#         fake_core_name = extract_core_name(fake_video_name)
#         if real_core_name == fake_core_name:
#             print(f"Match found: Real({real_video_name}) <-> Fake({fake_video_name})")  # Debugging line
#             return fake_video_name
#     return None


In [5]:
def extract_core_name(filename):
    """
    Extract the core name from the video filename that will be used for matching.
    Handles both complex (real) and simplified (fake) naming patterns.
    """
    # Remove file extension
    name_without_ext = filename.rsplit('.', 1)[0]
    
    # Split by underscores
    parts = name_without_ext.split('_')
    
    # Extract the action and emotion parts
    # Usually these are the last parts before any random identifiers
    if len(parts) >= 4:
        # For real videos: Get the action and emotion parts
        action_parts = []
        for part in parts[2:]:
            if any(emotion in part.lower() for emotion in ['happy', 'angry', 'sad', 'disgusted']):
                break
            action_parts.append(part)
        
        action = '_'.join(action_parts)
        emotion = next((part for part in parts if any(
            emotion in part.lower() for emotion in ['happy', 'angry', 'sad', 'disgusted']
        )), '')
        
        return f"{action}_{emotion}"
    
    return name_without_ext


In [6]:
def find_matching_fake_video(real_video_name, fake_video_dir):
    """
    Find the matching fake video for a given real video.
    Returns the filename of the matching fake video or None if no match is found.
    """
    real_core = extract_core_name(real_video_name)
    fake_videos = [f for f in os.listdir(fake_video_dir) if f.endswith('.mp4')]
    
    # For debugging
    print(f"Looking for match for real video: {real_video_name}")
    print(f"Extracted core: {real_core}")
    
    for fake_video in fake_videos:
        fake_core = extract_core_name(fake_video)
        # For debugging
        print(f"Comparing with fake video: {fake_video}")
        print(f"Fake core: {fake_core}")
        
        if real_core.lower() in fake_core.lower() or fake_core.lower() in real_core.lower():
            return fake_video
            
    return None

In [7]:
def debug_unmatched_files(real_video_dir, fake_video_dir):
    """
    Logs unmatched real and fake video files for debugging purposes.
    """
    import os

    real_videos = [f for f in os.listdir(real_video_dir) if f.endswith('.mp4')]
    fake_videos = [f for f in os.listdir(fake_video_dir) if f.endswith('.mp4')]

    real_core_names = {extract_core_name(f) for f in real_videos}
    fake_core_names = {extract_core_name(f) for f in fake_videos}

    unmatched_real = real_core_names - fake_core_names
    unmatched_fake = fake_core_names - real_core_names

    print("Unmatched real videos:")
    for core_name in unmatched_real:
        print(core_name)

    print("\nUnmatched fake videos:")
    for core_name in unmatched_fake:
        print(core_name)

In [8]:
def process_video_pair(real_video_dir, fake_video_dir, save_dir, video_id, max_frames=30):
    """
    Process a pair of matching real and fake videos.
    """
    real_videos = [f for f in os.listdir(real_video_dir) if f.endswith('.mp4')]
    processed_pairs = 0
    
    for real_video_name in real_videos:
        fake_video_name = find_matching_fake_video(real_video_name, fake_video_dir)
        
        if fake_video_name:
            real_path = os.path.join(real_video_dir, real_video_name)
            fake_path = os.path.join(fake_video_dir, fake_video_name)
            
            real_save_dir = os.path.join(save_dir, "real")
            fake_save_dir = os.path.join(save_dir, "fake")
            
            os.makedirs(real_save_dir, exist_ok=True)
            os.makedirs(fake_save_dir, exist_ok=True)
            
            print(f"Processing pair {processed_pairs + 1}:")
            print(f"Real: {real_video_name}")
            print(f"Fake: {fake_video_name}")
            
            extract_and_save_frames(real_path, real_save_dir, video_id + processed_pairs, max_frames)
            extract_and_save_frames(fake_path, fake_save_dir, video_id + processed_pairs, max_frames)
            
            processed_pairs += 1
        else:
            print(f"No matching fake video found for: {real_video_name}")
    
    return processed_pairs

In [9]:
def extract_and_save_frames(video_path, save_dir, video_id, max_frames=30):
    """Extract and immediately save frames from a video file."""
    if not os.path.exists(video_path):
        print(f"Warning: Video file not found: {video_path}")
        return 0

    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print(f"Warning: Could not open video: {video_path}")
        return 0

    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames == 0:
        print(f"Warning: Video has no frames: {video_path}")
        return 0

    # Calculate sampling interval
    sampling_interval = max(1, total_frames // max_frames)

    frame_count = 0
    saved_frames = 0

    while saved_frames < max_frames:
        success, frame = video.read()
        if not success:
            break

        if frame_count % sampling_interval == 0:
            frame_path = os.path.join(save_dir, f"video_{video_id:04d}_frame_{saved_frames:04d}.png")
            cv2.imwrite(frame_path, frame)
            saved_frames += 1

            # Clear memory
            del frame
            gc.collect()

        frame_count += 1

    video.release()
    return saved_frames


In [10]:
def process_videos_with_splits(real_video_dir, fake_video_dir, base_dir, max_frames=30):
    """Process videos and organize frames into train/test/validate splits."""
    # Get list of real videos
    real_videos = [f for f in os.listdir(real_video_dir) if f.endswith('.mp4')]
    
    # Randomly shuffle videos for splitting
    np.random.shuffle(real_videos)
    
    # Calculate split sizes (70% train, 15% test, 15% validate)
    total_videos = len(real_videos)
    train_size = int(0.7 * total_videos)
    test_size = int(0.15 * total_videos)
    
    # Split videos
    train_videos = real_videos[:train_size]
    test_videos = real_videos[train_size:train_size + test_size]
    validate_videos = real_videos[train_size + test_size:]
    
    # Process each split
    splits = {
        "train": train_videos,
        "test": test_videos,
        "validate": validate_videos
    }
    
    video_id = 0
    for split_name, videos in splits.items():
        print(f"\nProcessing {split_name} split ({len(videos)} videos)")
        for real_video_name in tqdm(videos):
            fake_video_name = find_matching_fake_video(real_video_name, fake_video_dir)
            
            if fake_video_name:
                # Setup paths
                real_path = os.path.join(real_video_dir, real_video_name)
                fake_path = os.path.join(fake_video_dir, fake_video_name)
                
                real_save_dir = os.path.join(base_dir, split_name, "real")
                fake_save_dir = os.path.join(base_dir, split_name, "fake")
                
                # Extract and save frames
                extract_and_save_frames(real_path, real_save_dir, video_id, max_frames)
                extract_and_save_frames(fake_path, fake_save_dir, video_id, max_frames)
                
                video_id += 1
            else:
                print(f"No matching fake video found for: {real_video_name}")

In [11]:
# def process_video_in_batches(real_video_dir, fake_video_dir, save_dir, batch_size=16, max_frames=30):
#     """Process videos in batches, extracting frames from matching real and fake videos."""
#     verify_paths(real_video_dir, fake_video_dir)
#     real_videos = [f for f in os.listdir(real_video_dir) if f.endswith('.mp4')]
#     # Iterate through real videos and find matching fake videos
#    # Iterate through real videos and find matching fake videos
#     matching_pairs = []
#     for real_video_name in real_videos:
#         real_video_path = os.path.join(real_video_dir, real_video_name)
#         matching_fake_videos = find_matching_fake_video(real_video_dir, fake_video_dir)  # Returns a list

#         if matching_fake_videos:
#             for fake_video in matching_fake_videos:  # Iterate through the matched fake videos
#                 fake_video_name = fake_video[0] if isinstance(fake_video, tuple) else fake_video
#                 matching_pairs.append((
#                     real_video_path,
#                     os.path.join(fake_video_dir, fake_video_name)
#                 ))

    
#     print(f"Found {len(matching_pairs)} matching video pairs.")

#     for i in range(0, len(matching_pairs), batch_size):
#         batch = matching_pairs[i:i + batch_size]
#         print(f"Processing batch {i // batch_size + 1} with {len(batch)} pairs.")

#         for video_id, (real_path, fake_path) in enumerate(batch):
#             real_save_dir = os.path.join(save_dir, "real")
#             fake_save_dir = os.path.join(save_dir, "fake")

#             os.makedirs(real_save_dir, exist_ok=True)
#             os.makedirs(fake_save_dir, exist_ok=True)

#             print(f"Processing Real: {real_path}, Fake: {fake_path}")

#             extract_and_save_frames(real_path, real_save_dir, video_id, max_frames)
#             extract_and_save_frames(fake_path, fake_save_dir, video_id, max_frames)

In [12]:
def main():
    # Set paths
    current_dir = os.getcwd()
    real_video_dir = os.path.join(current_dir, "/mnt/documents/Minor-Project/Datasets/FF++/real")
    fake_video_dir = os.path.join(current_dir, "/mnt/documents/Minor-Project/Datasets/FF++/fake")
    
    # Create directory structure
    base_dir = create_directory_structure()
    
    # Set parameters
    frames_per_video = 30
    
    # Process videos with train/test/validate splits
    process_videos_with_splits(real_video_dir, fake_video_dir, base_dir, frames_per_video)
    print("\nFrame extraction complete!")


In [13]:
# def main():
#     current_dir = os.getcwd()
#     real_video_dir = os.path.join(current_dir, "/mnt/documents/Minor-Project/Datasets/FF++/real")
#     fake_video_dir = os.path.join(current_dir, "/mnt/documents/Minor-Project/Datasets/FF++/fake")
    
#     base_dir = create_directory_structure()
#     frames_per_video = 30
    
#     # Process all videos
#     total_pairs = process_video_pair(real_video_dir, fake_video_dir, base_dir, 0, frames_per_video)
#     print(f"Successfully processed {total_pairs} video pairs")

In [17]:
if __name__ == "__main__":
    main()


Processing train split (140 videos)


  0%|          | 0/140 [00:00<?, ?it/s]

Looking for match for real video: 12__outside_talking_still_laughing.mp4
Extracted core: outside_talking_still_laughing_
Comparing with fake video: 09_03__talking_against_wall__8DTEGQ54.mp4
Fake core: _talking_against_wall__8DTEGQ54_
Comparing with fake video: 07_03__outside_talking_pan_laughing__IFSURI9X.mp4
Fake core: _outside_talking_pan_laughing__IFSURI9X_
Comparing with fake video: 03_07__walk_down_hall_angry__IFSURI9X.mp4
Fake core: _walk_down_hall_angry
Comparing with fake video: 06_04__outside_talking_still_laughing__ZK95PQDE.mp4
Fake core: _outside_talking_still_laughing__ZK95PQDE_


  1%|          | 1/140 [00:10<23:49, 10.28s/it]

Looking for match for real video: 07__walking_outside_cafe_disgusted.mp4
Extracted core: walking_outside_cafe_disgusted
Comparing with fake video: 09_03__talking_against_wall__8DTEGQ54.mp4
Fake core: _talking_against_wall__8DTEGQ54_
Comparing with fake video: 07_03__outside_talking_pan_laughing__IFSURI9X.mp4
Fake core: _outside_talking_pan_laughing__IFSURI9X_
Comparing with fake video: 03_07__walk_down_hall_angry__IFSURI9X.mp4
Fake core: _walk_down_hall_angry
Comparing with fake video: 06_04__outside_talking_still_laughing__ZK95PQDE.mp4
Fake core: _outside_talking_still_laughing__ZK95PQDE_
Comparing with fake video: 06_04__walking_outside_cafe_disgusted__ZK95PQDE.mp4
Fake core: _walking_outside_cafe_disgusted


...

Looking for match for real video: 14__exit_phone_room.mp4
Extracted core: exit_phone_room_
Comparing with fake video: 09_03__talking_against_wall__8DTEGQ54.mp4
Fake core: _talking_against_wall__8DTEGQ54_
Comparing with fake video: 07_03__outside_talking_pan_laughing__IFSURI9X.mp4
Fake core: _outside_talking_pan_laughing__IFSURI9X_
Comparing with fake video: 03_07__walk_down_hall_angry__IFSURI9X.mp4
Fake core: _walk_down_hall_angry
Comparing with fake video: 06_04__outside_talking_still_laughing__ZK95PQDE.mp4
Fake core: _outside_talking_still_laughing__ZK95PQDE_
Comparing with fake video: 06_04__walking_outside_cafe_disgusted__ZK95PQDE.mp4
Fake core: _walking_outside_cafe_disgusted
Comparing with fake video: 07_26__walk_down_hall_angry__FGNGC2GT.mp4
Fake core: _walk_down_hall_angry
Comparing with fake video: 06_27__talking_angry_couch__JOG5PB18.mp4
Fake core: _talking_angry
Comparing with fake video: 03_04__talking_angry_couch__T04P6ELC.mp4
Fake core: _talking_angry
Comparing with fake 

 97%|█████████▋| 29/30 [03:21<00:06,  6.12s/it]

Looking for match for real video: 06__outside_talking_still_laughing.mp4
Extracted core: outside_talking_still_laughing_
Comparing with fake video: 09_03__talking_against_wall__8DTEGQ54.mp4
Fake core: _talking_against_wall__8DTEGQ54_
Comparing with fake video: 07_03__outside_talking_pan_laughing__IFSURI9X.mp4
Fake core: _outside_talking_pan_laughing__IFSURI9X_
Comparing with fake video: 03_07__walk_down_hall_angry__IFSURI9X.mp4
Fake core: _walk_down_hall_angry
Comparing with fake video: 06_04__outside_talking_still_laughing__ZK95PQDE.mp4
Fake core: _outside_talking_still_laughing__ZK95PQDE_


100%|██████████| 30/30 [03:28<00:00,  6.96s/it]


Frame extraction complete!
